In [81]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import numpy as np
import re
import datetime
import random
from time import sleep, time
from IPython.core.display import clear_output
import pandas_profiling as pp

In [83]:
def xpath_text(xpath):
    try: return driver.find_element_by_xpath(xpath).text.replace('\n',' ').replace(';',',').replace('"','').strip()
    except: return ''

def clean(text,to_remove):
    if text == '':
        return None
    else:
        try: return re.search(f'(?<={to_remove}).*',text).group(0).replace('\u3000',' ').replace('\u25cf',' ').strip()
        except: return text.replace(to_remove,'',1).replace('\u3000',' ').replace('\u25cf',' ').strip()

def to_int(text):
    try: return int(text.replace('.',''))
    except: return None

def to_date(text):
    try: return datetime.datetime.strptime(text,'%d/%m/%Y').date()
    except: return None

In [97]:
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(executable_path=r'C:\Users\user\Notebooks\chromedriver.exe',options=chrome_options)

In [98]:
estado = [] # Estado de la oferta
trabajo = [] # Datos del trabajo: Ministerio, Institución, Cargo, N° Vacantes, Área, Región, Ciudad, Tipo de vacante, Renta bruta, Objetivo
formacion = [] # Formación educacional
requisito = [] # Requisitos generales o específicos
especializacion = [] # Especialización o capacitación
experiencia = [] # Experiencia sector público o privado
competencia = [] # Competencias
criterio = [] # Mecanismo de postulación - Proceso de selección
documento = [] # Documentos requeridos para postular
fecha = [] # Fechas de postulación, selección y finalización
condicion = [] # Condiciones generales para la postulación

url = "https://www.empleospublicos.cl/pub/convocatorias/avisopizarronficha.aspx?i="
start_time = time()
request = 0
init = 54000 # id de inicio
delta = 1180 # cantidad de registros a buscar

for i in range(init,init+delta): #54943 es el último id al 30/10/19
    # sleep(0.01*random.random())
    clear_output(wait = True)
    request += 1
    driver.get(url+str(i))
    
    estado.append(xpath_text('//*[@id="panelEstado"]'))
    formacion.append(xpath_text('//*[@id="lblFormacion"]'))
    trabajo.append([xpath_text('//*[@id="lblAvisoTrabajoDatos"]/ul/li['+str(j)+']') for j in range(1,11)])
    requisito.append(xpath_text('//*[@id="lblRequisitos"]'))
    especializacion.append(xpath_text('//*[@id="lblEspecializaciones"]'))
    experiencia.append(xpath_text('//*[@id="lblExperiencias"]'))
    competencia.append(xpath_text('//*[@id="lblCompetencias"]'))
    criterio.append(xpath_text('//*[@id="lblCriterio"]'))
    documento.append(xpath_text('//*[@id="lblContexto"]'))
    fecha.append([xpath_text('//*[@id="lblCalendarizacion"]/ul/li[1]/table/tbody/tr['+str(j)+']/td[2]') for j in range(3,8,2)])
    condicion.append(xpath_text('//*[@id="lblCalendarizacion"]/ul/li[5]'))
    
    print(f'Iteración: {i}; Frecuencia: {request/(time()-start_time)} iteraciones/s')
    print(f'Tiempo total: {str(datetime.timedelta(seconds=time()-start_time))}')
driver.quit()

Iteración: 55179; Frecuencia: 1.0955875906294823 iteraciones/s
Tiempo total: 0:17:57.047614


In [99]:
df = pd.DataFrame({
    'id': [i for i in range(init,init+delta)],
    'Estado': estado,
    'Ministerio': [clean(row[0],'Ministerio') for row in trabajo],
    'Institución': [clean(row[1],'Instituci.n / Entidad') for row in trabajo], #luego /TRIM, luego RTRIM
    'Cargo': [clean(row[2],'Cargo') for row in trabajo],
    'Vacantes': [to_int(clean(row[3],'Nº de Vacantes')) for row in trabajo],
    'Área': [clean(row[4].replace('/','/ '),'.rea de Trabajo') for row in trabajo],
    'Región': [clean(row[5],'Regi.n') for row in trabajo],
    'Ciudad': [clean(row[6],'Ciudad') for row in trabajo],
    'Tipo vacante': [clean(row[7],'Tipo de Vacante') for row in trabajo],
    'Renta bruta': [to_int(clean(row[8],'Renta Bruta')) for row in trabajo],
    'Objetivo': [clean(row[9],'Objetivo del Cargo') for row in trabajo],
    'Formacion': [clean(row, 'Formaci.n Educacional') for row in formacion],
    'Requisito': [clean(row,'Requisitos Generales / Específicos') for row in requisito],
    'Especializacion': [clean(row,'Especializaci.n y/o Capacitación') for row in especializacion],
    'Experiencia': [clean(row,'Experiencia sector público / sector privado') for row in experiencia],
    'Competencia': [clean(row,'Competencias') for row in competencia],
    'Mecanismo postulación': criterio,
    'Documento requerido': [clean(row,'Documentos requeridos para postular') for row in documento],
    'Fecha postulación inicio': [to_date(row[0][:row[0].find('-')]) for row in fecha],
    'Fecha postulación fin': [to_date(row[0][row[0].find('-')+1:]) for row in fecha],
    'Fecha selección inicio': [to_date(row[1][:row[1].find('-')]) for row in fecha],
    'Fecha selección fin': [to_date(row[1][row[1].find('-')+1:]) for row in fecha],
    'Fecha finalización inicio': [to_date(row[2][:row[2].find('-')]) for row in fecha],
    'Fecha finalización fin': [to_date(row[2][row[2].find('-')+1:]) for row in fecha],
    'Condicion': [clean(row,'Condiciones generales') for row in condicion]
    })

In [100]:
writer = pd.ExcelWriter(r'C:\Users\user\Documents\Proyectos\Empleos Públicos\Data_EE_54700_55179.xlsx',
                        engine='xlsxwriter',options={'strings_to_urls':False})
df.to_excel(writer,index=False)
writer.close()

In [95]:
pp.ProfileReport(df)